In [1]:
#import the required libraries
import cv2
import os
import numpy as np
import pandas as pd
from keras.models import load_model

#pygame is a python package which is helpful for playing the sounds in the background
from pygame import mixer

pygame 2.4.0 (SDL 2.26.4, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [7]:
#load the sound
mixer.init()
sound = mixer.Sound('alarm.wav')
#haarcascade classifier
face = cv2.CascadeClassifier(r'D:\computer_vision\Drowsiness_detection\haarcascade_frontalface_default.xml')
leye = cv2.CascadeClassifier(r'D:\computer_vision\Drowsiness_detection\haarcascade_lefteye_2splits.xml')
reye = cv2.CascadeClassifier(r'D:\computer_vision\Drowsiness_detection\haarcascade_righteye_2splits.xml')

label=['Close','Open']
#pretrained model 
model = load_model(r'D:\computer_vision\Drowsiness_detection\cnnCat2.h5')
path = os.getcwd()
cap = cv2.VideoCapture(r'D:\computer_vision\Drowsiness_detection\istockphoto-618279894-640_adpp_is.mp4')

count=0
score=0
rpred=[99]
lpred=[99]

while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2]

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    faces = face.detectMultiScale(gray,minNeighbors=5,scaleFactor=1.1,minSize=(25,25))
    left_eye = leye.detectMultiScale(gray)
    right_eye = reye.detectMultiScale(gray)
    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,255,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (0,0,255) , 1 )

    for (x,y,w,h) in right_eye:
        r_eye=frame[y:y+h,x:x+w]
        count=count+1
        r_eye = cv2.cvtColor(r_eye,cv2.COLOR_BGR2GRAY)
        r_eye = cv2.resize(r_eye,(24,24))
        r_eye= r_eye/255
        r_eye= r_eye.reshape(24,24,-1)
        r_eye = np.expand_dims(r_eye,axis=0)
        rpred = np.argmax(model.predict(r_eye))
        if(rpred==1):
            label='Open'
        if(rpred==0):
            label='Closed'
        break

    for (x,y,w,h) in left_eye:
        l_eye=frame[y:y+h,x:x+w]
        count=count+1
        l_eye = cv2.cvtColor(l_eye,cv2.COLOR_BGR2GRAY)
        l_eye = cv2.resize(l_eye,(24,24))
        l_eye= l_eye/255
        l_eye=l_eye.reshape(24,24,-1)
        l_eye = np.expand_dims(l_eye,axis=0)
        lpred = np.argmax(model.predict(l_eye))
        if(lpred==1):
            label='Open'
        if(lpred==0):
            label='Closed'
        break

    if(rpred==0 and lpred==0):
        score=score+1
        cv2.putText(frame,"Closed",(10,height-20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1)
    else:
        score=score-1
        cv2.putText(frame,"Open",(10,height-20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1)

    if(score<0):
        score=0
    cv2.putText(frame,'Score:'+str(score),(100,height-20), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(255,255,255),1)
    if(score>7):
        cv2.putText(frame,"ALERT!",(10,height-50), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1,(0,0,255),1)
        
        try:
            sound.play()
        except: # isplaying = False
            pass

        cv2.rectangle(frame,(0,0),(width,height),(0,0,255),3)
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

1/1 [==============================] - 0s 47ms/step
